In [1]:
!pip install transformers chromadb langchain langchain_community pypdf openai tiktoken rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.9/309.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Create Vector Store
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Agent
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.environ["OPENAI_API_KEY"] = "sk-proj-Dl8wLnwjSTHxeSdG8ep2T3BlbkFJycawnHY6xXlOsovmKH97"

In [5]:
# tokenizer = AutoTokenizer.from_pretrained("BioMistral/BioMistral-7B")
# model = AutoModel.from_pretrained("BioMistral/BioMistral-7B")
llm = OpenAI(temperature=0.1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [7]:
# embeddings = SentenceTransformerEmbeddings(model_name="neuml/pubmedbert-base-embeddings-matryoshka")
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="neuml/pubmedbert-base-embeddings-matryoshka",
                                           model_kwargs={"device": "cuda"},
                                           encode_kwargs={"device": "cuda", "batch_size": 100})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.43k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
def create_vector_db(path,uploaded_documents=1):
  DATA_DIR = path
  documents = []

  if uploaded_documents == 1:
    for file in os.listdir(DATA_DIR):
      if file.endswith('.pdf'):
        print(os.path.join(DATA_DIR, file))
        pdf_path = os.path.join(DATA_DIR, file)
        loader = PyPDFLoader(pdf_path, extract_images=True)
        documents.extend(loader.load())
  else:
    loader = PyPDFLoader(path, extract_images=True)
    documents.extend(loader.load())

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
  chunked_documents = text_splitter.split_documents(documents)
  print(chunked_documents)
  client = chromadb.Client()
  print(client.list_collections())
  if client.list_collections():
    print("Collection already exists")
  else:
    print("Collection created")
    consent_collection = client.create_collection("vector_database")
  vectordb = Chroma.from_documents(
          documents=chunked_documents,
          embedding=embeddings,
          persist_directory="/content/drive/My Drive/LLM Project/VectorDataBase/chroma_store/")
  vectordb.persist()

  return vectordb

In [9]:
docsearch = create_vector_db('/content/drive/MyDrive/data')
#docsearch = create_vector_db('https://arxiv.org/pdf/2402.12352.pdf',uploaded_documents=0)

/content/drive/MyDrive/data/mRNA Vaccine.pdf
/content/drive/MyDrive/data/Parkinsons.pdf
[Document(page_content='Citation: Wei, L.; Dong, C.; Zhu, W.;\nWang, B.-Z. mRNA Vaccine\nNanoplatforms and Innate Immunity.\nViruses 2024 ,16, 120. https://\ndoi.org/10.3390/v16010120\nAcademic Editor: Luis Martinez-\nSobrido\nReceived: 27 December 2023\nRevised: 8 January 2024\nAccepted: 12 January 2024\nPublished: 14 January 2024', metadata={'source': '/content/drive/MyDrive/data/mRNA Vaccine.pdf', 'page': 0}), Document(page_content='Published: 14 January 2024\nCopyright: © 2024 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed under the terms and\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/\n4.0/).\nviruses \nReview', metadata={'source': '/content/drive/MyDrive/data/mRNA Vaccine.pdf', 'page': 0}), Document(page_content='4.0/).\nviruses \nReview\nmRNA Vaccine Nanoplatforms and Inn

In [10]:
researchPapers = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [11]:
tools = [
    Tool(
        name = "Biomedical Research Papers",
        func=researchPapers.run,
        description="useful for when you need to answer questions about any biomedical research paper"
    )
]

In [12]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [16]:
agent.run("What are the advantages of mRNA delivery platform?")



> Entering new AgentExecutor chain...
 I should look for research papers that discuss mRNA delivery platforms
Action: Biomedical Research Papers
Action Input: "mRNA delivery platform"
Observation:  Recent advances in material sciences have yielded promising delivery platforms that facilitate the efficient delivery of mRNA, such as PbAEs and lipid-mRNA and polymer-based mRNA nanoparticles.
Thought: This is a good starting point, but I should also look for specific advantages mentioned in the papers
Action: Biomedical Research Papers
Action Input: "mRNA delivery platform advantages"
Observation:  The advantages of mRNA delivery platforms include efficient delivery of mRNA, protection of mRNA during endocytosis and endosome escape, and the potential to exert precise control and program over immune responses.
Thought: This is helpful, but I should also consider any potential limitations or challenges mentioned in the papers
Action: Biomedical Research Papers
Action Input: "mRNA delivery 

'The advantages of mRNA delivery platforms include efficient delivery of mRNA, protection of mRNA during endocytosis and endosome escape, and the potential to exert precise control and program over immune responses. However, there are also limitations such as the challenge of effectively delivering mRNA to cells in vivo and the need for further development of suitable delivery platforms. Additionally, the incomplete understanding of immune responses triggered by mRNA vaccines may also be a limitation.'

In [17]:
agent.run("What are the symptoms of Parkinson's disease?")



> Entering new AgentExecutor chain...
 I should use biomedical research papers to find information about Parkinson's disease.
Action: Biomedical Research Papers
Action Input: Parkinson's disease symptoms
Observation:  The symptoms of Parkinson's disease include motor symptoms such as tremor, rigidity, sluggishness, and unsteadiness, as well as nonmotor symptoms such as depression, hypotension, sleep disturbances, irritability, and constipation. These symptoms appear gradually and may be preceded by nonmotor symptoms. Tremor is a common motor symptom and is often the initial sign of the disease. Parkinson's disease can also lead to other motor disorders such as cerebral dyskinesia and dystonia. Due to the difficulty of clinical diagnosis, Parkinson's disease is often misdiagnosed, highlighting the need for better detection methods. Treatment for Parkinson's disease is available, but it is a progressive ailment that affects the neurological system and nerve-controlled areas of the body

"The symptoms of Parkinson's disease include motor symptoms such as tremor, rigidity, sluggishness, and unsteadiness, as well as nonmotor symptoms such as depression, hypotension, sleep disturbances, irritability, and constipation."

In [18]:
agent.run("Can you summarize the Parkinson's research paper in about 100 words")



> Entering new AgentExecutor chain...
 I should use the Biomedical Research Papers tool to find the Parkinson's research paper and then read through it to summarize it.
Action: Biomedical Research Papers
Action Input: "Parkinson's research paper"
Observation:  The context provided is from a research paper on Parkinson's disease, specifically focusing on its pathogenesis, clinical scenario, diagnosis, and treatment. The paper discusses the lack of attention given to the health of Parkinson's patients and the clinical diagnosis of the disease through reviewing the patient's medical history. It also mentions the identification of alpha-synuclein as a key neuronal protein in the diagnosis of Parkinson's disease. The paper is published in the journal 3 Biotech in 2023.
Thought: I should read through the paper and take notes on the key points to summarize it.
Action: Read through the paper and take notes
Action Input: Parkinson's research paper
Observation: Read through the paper and take 

"The Parkinson's research paper discusses the pathogenesis, clinical scenario, diagnosis, and treatment of the disease. It highlights the lack of attention given to the health of Parkinson's patients and the role of alpha-synuclein in diagnosis. Techniques such as NMR spectroscopy and fluorescence measurement are also mentioned in the paper."

In [19]:
# Chatbot

In [20]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain
from transformers import  AutoModelForCausalLM, AutoTokenizer

In [21]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [22]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [24]:
agent_chain.run(input="How does immune dysregulation contribute to the pathophysiology of Parkinson's disease??")



> Entering new AgentExecutor chain...
Thought: I should consult biomedical research papers to find the answer.
Action: Biomedical Research Papers
Action Input: "Immune dysregulation and Parkinson's disease"
Observation:  There is evidence that neuroinflammation and immune dysregulation play a role in the development and progression of Parkinson's disease. Studies have shown an interaction between neuroinflammation and Parkinson's disease pathogenic pathways, and pro-inflammatory cytokines have been found to be increased in the brains of Parkinson's disease patients. It is important to diagnose the disease early using nanobiosensors and future research and treatments should focus on dysfunctional brain systems.
Thought: I now know the final answer.
Final Answer: Immune dysregulation contributes to the pathophysiology of Parkinson's disease through neuroinflammation and interactions with pathogenic pathways. Early diagnosis using nanobiosensors is important and future research should f

"Immune dysregulation contributes to the pathophysiology of Parkinson's disease through neuroinflammation and interactions with pathogenic pathways. Early diagnosis using nanobiosensors is important and future research should focus on dysfunctional brain systems."